## **TESTING:** etl_21 -- TSDB

In [1]:
import os
import pandas as pd
import pyarrow
import pytz
from datetime import datetime, timedelta
import pyarrow.feather as feather
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from pandasql import sqldf
import sqlite3
import argparse
import shutil
import psycopg2
from sqlalchemy import create_engine


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


### **CONENCTION**

In [2]:
CONNECTION = "postgres://tsdbadmin:>X>#h2lWqXESlyGd}mj2NPDlB@esd7mq3z84.dts890uzaz.tsdb.cloud.timescale.com:37281/tsdb?sslmode=require"

Using psycopg2

In [3]:
conn = psycopg2.connect(CONNECTION)

Using SQLAlchemy

In [5]:
# conn = create_engine(CONNECTION)

Convert History to SCD

# **CHECKS** Scope

In [4]:
l_check_prices = ['61277.43000000',
 '61111.20000000',
 '61268.29000000',
 '61132.62000000',
 '61299.90000000',
 '61058.75000000',
 '61056.20000000',
 '61223.36000000',
 '61065.03000000',
 '61227.34000000',
 '61056.70000000',
 '61237.96000000',
 '61170.44000000',
 '61136.34000000',
 '61222.59000000',
 '61111.51000000',
 '61227.27000000',
 '61158.14000000',
 '61303.54000000',
 '61047.88000000']

# **CHECKS** Database

### Tables

In [6]:
import psycopg2

def list_tables(conn):
    # Create a cursor object
    cursor = conn.cursor()

    # Execute the query to list all tables
    query = """
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_type = 'BASE TABLE' and table_schema = 'public'
    ORDER BY table_schema, table_name;
    """
    cursor.execute(query)

    # Fetch all the results
    tables = cursor.fetchall()

    # Print the results
    for table in tables:
        # print(table)
        print(f"{table[0]}.{table[1]}")

    # Close the cursor and connection
    cursor.close()
    # conn.close()

list_tables(conn)


public.l2_history_ask
public.l2_history_bid
public.l2_snapshot_ask
public.l2_snapshot_bid
public.logs
public.update_i


Convert History to SCD

### Snapshot

In [ ]:
# s01_02.columns

In [ ]:
# Define the query
sql_s01 = 'SELECT * FROM l2_snapshot_ask'

# Execute the query and load the data into a pandas DataFrame
s01_01 = pd.read_sql_query(sql_s01, conn)
s01_02 = s01_01[s01_01['price'].isin(l_check_prices)].copy()

display(s01_02)

### History

In [ ]:
# Define the query
sql_h01 = 'SELECT * FROM l2_history_ask'

# Execute the query and load the data into a pandas DataFrame
h01_01 = pd.read_sql_query(sql_h01, conn)
h01_02 = h01_01[h01_01['price'].isin(l_check_prices)]
display(h01_02)

In [ ]:
# Define the query
sql_h01 = 'SELECT * FROM l2_history_ask'

# Execute the query and load the data into a pandas DataFrame
h01_01 = pd.read_sql_query(sql_h01, conn)
h01_02 = h01_01[h01_01['price'].isin(l_check_prices)]
display(h01_02)

# **CHECKS** Uniqueness

In [ ]:

    db_snap_02 = pd.read_sql_query(
        """
        SELECT  lastupdateid
        ,count(distinct side) as D_Side
        FROM l2_snapshot
        GROUP BY lastupdateid 
        """
        , conn)
    db_snap_02.D_Side.value_counts()    


# **CHECKS** Logs

In [9]:
# Connect to the SQLite database
# Define the query
sql_logs = 'SELECT * FROM logs'

# Execute the query and load the data into a pandas DataFrame
db_logs_01 = pd.read_sql_query(sql_logs, conn)

# Display the DataFrame
print(db_logs_01.level.value_counts())
print("---------------------------------------------------------------")
# display(db_logs_01)

level
INFO     63
ERROR     2
Name: count, dtype: int64
---------------------------------------------------------------


/tmp/ipykernel_109297/2746400065.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  db_logs_01 = pd.read_sql_query(sql_logs, conn)


In [11]:
display(db_logs_01)
# display(db_logs_01.tail(15))
# display(db_logs_01[db_logs_01['level']=='ERROR'].tail(10))

,timestamp,level,p_db_ver,message
0,2024-05-27 21:37:29.968269,INFO,20_load_snapshot.py,***main*** LOADING to Database
1,2024-05-27 21:37:30.003652,INFO,20_load_snapshot.py,***f_ETL_Init_01*** Read in file: ../test/data/001_raw/l2s_BTCUSDT_G002_00502_20240511_1315.feather
2,2024-05-27 21:37:30.026516,INFO,20_load_snapshot.py,***f_ETL_Init_01*** Read in # rows: 10000
3,2024-05-27 21:37:30.049475,INFO,20_load_snapshot.py,***f_ETL_Init_01*** l2_snapshot_01
4,2024-05-27 21:37:30.317496,INFO,20_load_snapshot.py,***copy_from_dataframe*** to csv to database
5,2024-05-27 21:37:30.747431,INFO,20_load_snapshot.py,***copy_from_dataframe*** to csv to database
6,2024-05-27 21:37:31.109424,INFO,20_load_snapshot.py,***copy_from_dataframe*** to csv to database
7,2024-05-27 21:37:31.563734,INFO,20_load_snapshot.py,***copy_from_dataframe*** to csv to database
8,2024-05-27 21:37:32.043552,INFO,20_load_snapshot.py,***main*** MOVE l2s_BTCUSDT_G002_00502_20240511_1315.feather file to processed folder
9,2024-05-27 21:37:49.834761,INFO,30_load_update_20.py,***main*** Starting the update process


# **DEEP DIVE** 

In [13]:
p_dir_updates = "../test/data/001_raw/"
filename = "l2_BTCUSDT_TEST__00502_20240511_1936.feather"
l2_update_02 = pd.read_feather(os.path.join(p_dir_updates, filename))



In [21]:
p_dir_updates = "../test/data/001_raw/"
filename_s = "l2s_BTCUSDT_G002_00502_20240511_1315.feather"
l2_snap_02 = pd.read_feather(os.path.join(p_dir_updates, filename_s))


In [14]:
l2_update_02.head()

,ticker,side,price,quantity,timestamp,lastupdateid,timestampevent
940,BTCUSDT,ASK,61227.27000000,0.00000000,2024-05-11 12:36:32.388487|EDT-0400,46922716597,1715445393250
941,BTCUSDT,ASK,61227.34000000,0.00000000,2024-05-11 12:36:32.388487|EDT-0400,46922716597,1715445393250
3784,BTCUSDT,ASK,61223.36000000,0.00000000,2024-05-11 12:37:24.538174|EDT-0400,46922726066,1715445445253
3822,BTCUSDT,ASK,61227.27000000,0.01069000,2024-05-11 12:37:24.538174|EDT-0400,46922726066,1715445445253
4278,BTCUSDT,ASK,61227.27000000,0.00000000,2024-05-11 12:37:26.401616|EDT-0400,46922726524,1715445447251


In [28]:
from utils import *
from utils_tsdb import *

In [29]:
l2_update_02[['ts_e_est']] = l2_update_02['timestampevent'].apply(f_convert_ts_to_dt_02)
l2_update_03 =l2_update_02[(l2_update_02['price']=='61227.27000000') & (l2_update_02['side']=='ASK')]
display(l2_update_03)

,ticker,side,price,quantity,timestamp,lastupdateid,timestampevent,ts_e_est
940,BTCUSDT,ASK,61227.27000000,0.00000000,2024-05-11 12:36:32.388487|EDT-0400,46922716597,1715445393250,2024-05-11 12:36:33.250000
3822,BTCUSDT,ASK,61227.27000000,0.01069000,2024-05-11 12:37:24.538174|EDT-0400,46922726066,1715445445253,2024-05-11 12:37:25.253000
4278,BTCUSDT,ASK,61227.27000000,0.00000000,2024-05-11 12:37:26.401616|EDT-0400,46922726524,1715445447251,2024-05-11 12:37:27.251000
207303,BTCUSDT,ASK,61227.27000000,0.00039000,2024-05-11 13:06:10.472852|EDT-0400,46923287585,1715447171334,2024-05-11 13:06:11.334000
209304,BTCUSDT,ASK,61227.27000000,0.00000000,2024-05-11 13:06:27.476111|EDT-0400,46923292831,1715447188335,2024-05-11 13:06:28.335000


In [25]:
l2_snap_03 = l2_snap_02[(l2_snap_02['Price']=='61227.27000000') & (l2_snap_02['Side']=='ASK')]
display(l2_snap_03)

,Ticker,Side,Price,Quantity,Timestamp,LastUpdateId
8521,BTCUSDT,ASK,61227.27000000,0.00051000,2024-05-11 13:15:50.905435,46916616325


In [27]:
display(db_logs_01[db_logs_01['level']=='ERROR'].tail(10))

,timestamp,level,p_db_ver,message
21,2024-05-27 21:37:50.589328,ERROR,30_load_update_20.py,"***f_L01_update*** Error updating l2_snapshot_ask and l2_history_ask: duplicate key value violates unique constraint ""l2_history_ask_pkey""\nDETAIL: Key (price, side, scd_from_date)=(61227.27000000, ASK, 2024-05-11 12:37:25.253+00) already exists.\n"
63,2024-05-27 21:37:54.327107,ERROR,30_load_update_20.py,"***f_L01_update*** Error updating l2_snapshot_bid and l2_history_bid: duplicate key value violates unique constraint ""l2_history_bid_pkey""\nDETAIL: Key (price, side, scd_from_date)=(61158.14000000, BID, 2024-05-11 12:47:58.284+00) already exists.\n"
